In [1]:
import requests
import pandas as pd
import numpy as np
import warnings
import datetime
warnings.filterwarnings('ignore')
# ============上市股票df============
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]

# ============上櫃股票df============
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]

# ============上市股票代號+.TW============
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)

# ============上櫃股票代號+.TWO============
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)

# ============concate全部股票代號============
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)


     有價證券名稱 市場別    產業別 公開發行/上市(櫃)/發行日
ID                                   
1101     台泥  上市   水泥工業     1962/02/09
1102     亞泥  上市   水泥工業     1962/06/08
1103     嘉泥  上市   水泥工業     1969/11/14
1104     環泥  上市   水泥工業     1971/02/01
1108     幸福  上市   水泥工業     1990/06/06
...     ...  ..    ...            ...
9949     琉園  上櫃  文化創意業     2003/11/21
9950    萬國通  上櫃   塑膠工業     2004/02/17
9951     皇田  上櫃   電機機械     2003/12/18
9960    邁達康  上櫃   運動休閒     2004/12/06
9962     有益  上櫃   鋼鐵工業     2006/07/10

[1839 rows x 4 columns]


## 全部策略選中股票

In [26]:
date = '2024-11-22'
filename = f'daily_stock_summary_{date}_with_template'
path = r'C:\Users\User\Desktop\stock\全個股條件篩選\\' + filename + '.xlsx'
# path = f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/全個股條件篩選/{date}.xlsx'
df = pd.read_excel(path)
df['ID'] = df['ID'].apply(lambda x: str(x).split('.')[0])
df = df.set_index('ID')
df['市場別'] = None
df.drop(df[df['產業別']==0].index.values, inplace=True)
for i in df.index:
    df.loc[i, '市場別'] = allstock_info.loc[i, '市場別']
all_df = df.copy()
T5_id = all_df.loc[all_df['T5']].index.values
T5_2_id = all_df.loc[all_df['T5-2']].index.values 
T6_id = all_df.loc[all_df['T6']].index.values
T11_id = all_df.loc[all_df['T11']].index.values
T21_id = all_df.loc[all_df['T21']].index.values
TM = all_df.loc[all_df['TM']].index.values
T5_2_id = [i for i in T5_2_id if i not in T5_id]
T6_id = [i for i in T6_id if all([i not in T5_id, i not in T5_2_id])]
T11_id = [i for i in T11_id if all([i not in T5_id, i not in T5_2_id, i not in T6_id])]
T21_id = [i for i in T21_id if all([i not in T5_id, i not in T5_2_id, i not in T6_id, i not in T11_id])]
TM = [i for i in TM if all([i not in T5_id, i not in T5_2_id, i not in T6_id, i not in T11_id, i not in T21_id])]
template = ['T5', 'T5-2', 'T6', 'T11', 'T21', 'TM']
ids = [T5_id, T5_2_id, T6_id, T11_id, T21_id, TM]
all_text = ''
for i, t in enumerate(template):
    text = ''
    all_text += f'###{t}\n'
    for id in ids[i]:
        if df.loc[id, '市場別'] == '上櫃':
            text += f'TPEX:{id}\n'
        elif df.loc[id, '市場別'] == '上市':
            text += f'TWSE:{id}\n'
        # create and save to txt
    all_text += text
    
with open(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/tradingview觀察清單/{date}_all.txt', 'w', encoding='utf-8') as f:
    f.write(all_text)
print(f'{date}_all.txt saved')


2024-11-22_all.txt saved


## 每日新增的股票

In [33]:
a= [1,2,3]
a.insert(0, 'sad')
print(a)


['sad', 1, 2, 3]


In [25]:
import pandas as pd
import numpy as np
# ============讀今日和昨日的股票Excel============

path = r'C:\Users\User\Desktop\stock\全個股條件篩選\\' + filename + '.xlsx'
new_date = '2024-11-21'
new_filename = f'daily_stock_summary_{new_date}_with_template'
new_path = r'C:\Users\User\Desktop\stock\全個股條件篩選\\' + new_filename + '.xlsx'
old_date = '2024-11-20'
old_filename = f'daily_stock_summary_{old_date}_with_template' 
old_path = r'C:\Users\User\Desktop\stock\全個股條件篩選\\' + old_filename + '.xlsx'
df_new = pd.read_excel(new_path)
df_old = pd.read_excel(old_path)
df_new['ID'] = df_new['ID'].apply(lambda x: str(x).split('.')[0])
df_old['ID'] = df_old['ID'].apply(lambda x: str(x).split('.')[0])
df_new = df_new.set_index('ID')
df_old = df_old.set_index('ID')
df_new.drop(df_new[df_new['產業別']==0].index.values, inplace=True)
df_old.drop(df_old[df_old['產業別']==0].index.values, inplace=True)
df_new['市場別'] = None
df_old['市場別'] = None

# ============今日和昨日股票的ID============
df_same_ID = df_new.loc[df_new.index.isin(df_old.index.values)].index.values
print(f'共有{len(df_same_ID)}支股票')
df_old = df_old.loc[df_same_ID]
df_new = df_new.loc[df_same_ID]
old_T5_ID = df_old.loc[df_old['T5']].index.values
old_T5_2_ID = df_old.loc[df_old['T5-2']].index.values
old_T6_ID = df_old.loc[df_old['T6']].index.values
old_T11_ID = df_old.loc[df_old['T11']].index.values
new_T5_ID = df_new.loc[df_new['T5']].index.values
new_T5_2_ID = df_new.loc[df_new['T5-2']].index.values
new_T6_ID = df_new.loc[df_new['T6']].index.values
new_T11_ID = df_new.loc[df_new['T11']].index.values
new_T21_ID = df_new.loc[df_new['T21']].index.values
old_T21_ID = df_old.loc[df_old['T21']].index.values

# ============新增和刪除股票的ID============
T5_add_ID = np.sort([i for i in new_T5_ID if i not in old_T5_ID])
print(new_T5_ID)
print(old_T5_ID)
T5_drop_ID = np.sort([i for i in old_T5_ID if i not in new_T5_ID])
T6_add_ID = np.sort([i for i in new_T6_ID if i not in old_T6_ID])
T6_drop_ID = np.sort([i for i in old_T6_ID if i not in new_T6_ID])
T11_add_ID = np.sort([i for i in new_T11_ID if i not in old_T11_ID])
T11_drop_ID = np.sort([i for i in old_T11_ID if i not in new_T11_ID])
T5_2_add_ID = np.sort([i for i in new_T5_2_ID if i not in old_T5_2_ID])
T5_2_drop_ID = np.sort([i for i in old_T5_2_ID if i not in new_T5_2_ID])
T21_add_ID = np.sort([i for i in new_T21_ID if i not in old_T21_ID])
T21_drop_ID = np.sort([i for i in old_T21_ID if i not in new_T21_ID])
# T5_drop_goodID = np.sort([i for i in T5_drop_ID if all([df_new.loc[i,'S250rate']>=75, df_new.loc[i, 'S20rate']>85])])
# T5_2_drop_goodID = np.sort([i for i in T5_2_drop_ID if all([df_new.loc[i,'ES250rate']>=75, df_new.loc[i, 'ES20rate']>85])])
T5_2_add_ID = [i for i in T5_2_add_ID if i not in T5_add_ID]
T6_add_ID = [i for i in T6_add_ID if all([i not in T5_add_ID, i not in T5_2_add_ID])]
T11_add_ID = [i for i in T11_add_ID if all([i not in T5_add_ID, i not in T5_2_add_ID, i not in T6_add_ID])]
T21_add_ID = [i for i in T21_add_ID if all([i not in T5_add_ID, i not in T5_2_add_ID, i not in T6_add_ID, i not in T11_add_ID])]

template = ['T5', 'T5-2', 'T6', 'T11', 'T21']
add = [T5_add_ID, T5_2_add_ID, T6_add_ID, T11_add_ID, T21_add_ID]
print(add)
all_txt = ''
for i, t in enumerate(template):
    all_txt += f'###{t}\n'
    text = ''
    for id in add[i]:
        id = str(id)
        if allstock_info.loc[id, '市場別'] == '上櫃':
            text += f'TPEX:{id}\n'
        elif allstock_info.loc[id, '市場別'] == '上市':
            text += f'TWSE:{id}\n'
        # create and save to txt
    all_txt += text
with open(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/tradingview觀察清單/{new_date}_all_add.txt', 'w', encoding='utf-8') as f:
    f.write(all_txt)
print(f'{new_date}-all_add.txt saved')

共有1743支股票
['8446' '8240' '8234' '6821' '6781' '6582' '6558' '6166' '5531' '5530'
 '5475' '5460' '5315' '5309' '5243' '4766' '4557' '3416' '3338' '3323'
 '3290' '3231' '3088' '3056' '3032' '3015' '2929' '2915' '2888' '2885'
 '2852' '2610' '2606' '2506' '2488' '2444' '2387' '2383' '2356' '2345'
 '2332' '2328' '2324' '2233' '1817' '1795' '1565' '1504' '1449' '1402'
 '1336' '1220']
['8446' '8240' '6821' '6781' '6606' '6510' '6166' '5530' '5488' '5315'
 '5309' '5243' '4577' '4557' '4129' '3499' '3416' '3323' '3231' '3088'
 '3056' '3032' '3015' '2915' '2888' '2885' '2852' '2610' '2606' '2506'
 '2444' '2387' '2345' '2332' '2328' '2324' '2233' '2069' '1817' '1795'
 '1565' '1504' '1463' '1449' '1402' '1336' '1220']
[array(['2356', '2383', '2488', '2929', '3290', '3338', '4766', '5460',
       '5475', '5531', '6558', '6582', '8234'], dtype='<U4'), ['1471', '1802', '2332', '2387', '2419', '2885', '2915', '3010', '3380', '4532', '6150', '6274', '6288', '8038', '8171', '8261'], ['2486', '3303', '49

In [ ]:
df = pd.read_excel(f'{path}{new_date}選股.xlsx', header=0, index_col=0)
df['ESS250diff'] = df['ES250rate'] - df['S250rate']
df['5MA_close_diff'] = (df['Adj Close'] - df['5MA'])/df['5MA']
df['200MA_close_diff'] = (df['Adj Close'] - df['200MA'])/df['200MA']
df['50MA_close_diff'] = (df['Adj Close'] - df['50MA'])/df['50MA']
## 5MA_close_diff > 0.05 and ESS250diff > 0.05
df = df[(df['5MA_close_diff'] < 0.05) & (df['ESS250diff'] > 10)
& (df['ES250 is 10D MAX'] != 1) & (df['ES50 is 10D MAX'] != 1) & (df['ES20 is 10D MAX'] != 1)
& (df['busness volume(億)'] > 0.8) & (df['year high sort'] == 1) & (df['200MA_close_diff'] < 0.8)
& (df['200MA_close_diff'] > 0.2) & (df['S250rate'] < 90) & (df['50MA_close_diff'] > 0.15)]
print(df)
stock_id = pd.read_excel(r'C:\Users\User\Desktop\StockInfoHub\others\產業別.xlsx')
industry = 'T5-2_test_1'
stock_id = df.index.values
text = ''
for id in stock_id:
    id = str(id)
    if allstock_info.loc[id, '市場別'] == '上櫃':
        text += f'TPEX:{id}\n'
    elif allstock_info.loc[id, '市場別'] == '上市':
        text += f'TWSE:{id}\n'
    # create and save to txt
with open(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/tradingview觀察清單/{new_date}_T31_all', 'w', encoding='utf-8') as f:
    f.write(text)
print(f'{industry}.txt saved')

## 股票ID轉tradingview格式

In [7]:
import pandas as pd
stock_id = pd.read_excel(r'C:\Users\User\Desktop\StockInfoHub\others\產業別.xlsx')
revenue_highest_df = pd.read_csv(r'C:\Users\User\Desktop\Excel\StockList.csv', encoding='utf-8-sig')
revenue_highest_id = revenue_highest_df.loc[revenue_highest_df['單月營收歷月排名'] == '1高', '代號']
industry = '202410營收創高股'
# stock_id = stock_id[industry]
# stock_id = stock_id.loc[stock_id != 0]
text = ''
for id in revenue_highest_id.values:
    id = str(id)
    try:
        if allstock_info.loc[id, '市場別'] == '上櫃':
            text += f'TPEX:{id}\n'
        elif allstock_info.loc[id, '市場別'] == '上市':
            text += f'TWSE:{id}\n'
    except:
        pass
    # create and save to txt
with open(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/tradingview觀察清單/{industry}.txt', 'w', encoding='utf-8') as f:
    f.write(text)
print(f'{industry}.txt saved')

202410營收創高股.txt saved
